In [1]:
london_loc = "51.520624794291884, -0.0870022262169723"

In [2]:
singapore_loc = "1.2862863534171092, 103.84868192263733"

In [3]:
san_francisco_loc = "37.79787988216447, -122.41049370727525"

In [4]:
import requests
import json
from dotenv import load_dotenv
import os
import pandas as pd
import pyjsonviewer
import geopandas 
import shapely
import pymongo as pym
from pymongo import MongoClient,GEOSPHERE
import shapely.geometry as shg

In [5]:
load_dotenv()
#Tenemos los tokens guardados en un .env en el visual studio code

True

In [6]:
tk_id = os.getenv("token_client_id")

In [7]:
tk_secret = os.getenv("token_client_secret")

In [8]:
git_tk = os.getenv("github_tk")

In [9]:
url = 'https://api.foursquare.com/v2/venues/explore'

In [10]:
#pyjsonviewer.view_data(json_data = LaVariableQueQuierasSinComillas)

In [11]:
def llamada_API(url, coordenadas, query):
    parametros = {
    "client_id": tk_id,
    "client_secret": tk_secret,
    "v": "20180323",
    "ll": f"{coordenadas}",
    "query": f"{query}",
    "limit": 200  
}
    response = requests.get(url, params=parametros).json()["response"]["groups"][0]["items"]
    return response

In [12]:
def escoger_datos(response):
    name = []
    type_ = []
    lat = []
    lng = []
    cc = []
    city = []
    adress = []
    postal_code = []
    for i in response:
        name.append(i["venue"]["name"])
    for i in response:
        type_.append(i["venue"]["categories"][0]["name"])
    for i in response:
        lng.append(i["venue"]["location"]["lng"])
    for i in response:
        lat.append(i["venue"]["location"]["lat"])
    for i in response:
        cc.append(i["venue"]["location"]["cc"])
    for i in response:
        adress.append(i["venue"]["location"]["address"])   
    for i in response:
        postal_code.append(i["venue"]["location"]["postalCode"])    
    response2 = [name , type_, cc, postal_code, adress,  lat, lng ]

    return response2

In [13]:
def crear_df(response2):
    df = pd.DataFrame(response2).T
    df1 = df.rename(columns = {0:"Name", 1:"Type", 2:"Country",  3:"Postal Code", 4:"Adress", 5:"Latitude", 6:"Longitude", })
    return df1

In [14]:
def get_geometry(df):
    df2 = geopandas.GeoDataFrame(df, geometry=geopandas.points_from_xy(df.Latitude, df.Longitude))
    return df2

In [15]:
def get_loc(df):
    df3 = df['geometry'].apply(lambda x:shapely.geometry.mapping(x))
    return pd.DataFrame(df3)

In [22]:
def df_final(url, coordenadas, query):
    a  = llamada_API(url, coordenadas, query)
    b = escoger_datos(a)
    c = crear_df(b)
    d = get_geometry(c)
    d["location"] = get_loc(d)
    df_terminado = d[["Name","Type","Country","Postal Code", "Adress","Latitude","Longitude","location"]]
    return df_terminado

In [23]:
df1 = df_final(url, london_loc, "Nursery School")

In [24]:
df1

,Name,Type,Country,Postal Code,Adress,Latitude,Longitude,location
0,Hatching Dragons,Nursery School,GB,EC1Y 0AA,1 Viscount Street,51.521599,-0.095356,"{'type': 'Point', 'coordinates': (51.521599411..."
1,Bright Horizons City Child Nursery and Preschool,Nursery School,GB,EC2Y 8AH,1 Bridgewater Square Barbican,51.521406,-0.095647,"{'type': 'Point', 'coordinates': (51.5214063, ..."
2,Hatching Dragons,Nursery School,GB,EC2Y 8AA,"Ferroners House, Shaftesbury Place, Aldersgate St",51.517948,-0.096538,"{'type': 'Point', 'coordinates': (51.517948, -..."
3,Bright Horizons Spitalfields Day Nursery and P...,Nursery School,GB,E1 6EA,21 Lamb Street,51.520314,-0.074974,"{'type': 'Point', 'coordinates': (51.520313793..."
4,Hopes and Dreams Montessori Nursery School,Nursery School,GB,EC1V 1LJ,339-341 City Road,51.53105,-0.102079,"{'type': 'Point', 'coordinates': (51.531049874..."
5,The Soulful Nutritionist,Nursery School,GB,EC1R 5HL,9 Coldbath Square,51.523818,-0.110315,"{'type': 'Point', 'coordinates': (51.5238179, ..."
6,Rosemary Works Big Play School,Nursery School,GB,N1 5PH,Branch Place,51.536926,-0.083277,"{'type': 'Point', 'coordinates': (51.536926463..."
7,Montessori by Busy Bees,Nursery School,GB,E1 0BH,"St George in the East Crypt West, 14 Cannon St...",51.51005,-0.059813,"{'type': 'Point', 'coordinates': (51.5100497, ..."
8,Bright Horizons Tabard Square Nursery and Pres...,Nursery School,GB,SE1 4NA,"10-12 Empire Square, Tabard Street",51.500406,-0.090973,"{'type': 'Point', 'coordinates': (51.5004061, ..."
9,Montessori by Busy Bees,Nursery School,GB,E1W 2RG,4 Wapping Lane,51.508951,-0.058398,"{'type': 'Point', 'coordinates': (51.5089513, ..."
